In [212]:
import pandas as pd
import sidetable
import regex as re
import numpy as np

In [213]:
df = pd.read_pickle("datos/attacks_pandas_8.pickle")

HIPOTESIS 

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

En este ejercicio nos queda por limpiar las columnas de age y species. Trabajaremos con el archivo generado en limpieza 1

In [214]:
df.sample()

,case_number,year,type,country,area,location,age,species_,date,mes,fatal_(y/n)_limpio,sex,country_
4443,1983.03.00,<NA>,NaN,nan,NaN,NaN,NaN,NaN,Mar-1983,NaN,N,F,NaN


1 Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified".
- Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.
- Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [215]:
patron_blanco = r".*[Ww](hite|HITE).*"
patron_tigre = r".*[Tt](iger|IGER).*"
patron_gris = r".*[Gg](rey|REY).*"
patron_limon = r".*[Ll](emon|EMON).*"
patron_toro = r".*[Bb](ull|ULL).*"

In [216]:
def clasificacion(col):
    col = str(col)
    if re.findall(patron_blanco, col):
        return "White Shark"
    elif re.findall(patron_tigre, col):
        return "Tiger Shark"
    elif re.findall(patron_gris, col):
        return "Grey Shark"
    elif re.findall(patron_limon, col):
        return "Lemon Shark"
    elif re.findall(patron_toro, col):
        return "Bull Shark"
    else:
        return "Unspecified"

In [217]:
df["species_"]= df["species_"].apply(clasificacion)

In [218]:
df.sample(2)

,case_number,year,type,country,area,location,age,species_,date,mes,fatal_(y/n)_limpio,sex,country_
5923,1951.09.29,<NA>,NaN,nan,NaN,NaN,NaN,Unspecified,29-Sep-1951,Sep,Y,F,NaN
1532,1700.00.00.b,1700,Unprovoked,france,Côte d'Azur,Antibes,NaN,White Shark,NaN,NaN,NaN,M,france


Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
- Edad en formato string
- Edades separadas por &, or, to, >
- Edades con ?

- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen.
- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.
- Por último cambiad el tipo de la columna de string a integer.


In [219]:
patron_num = "[0-99]+"

In [220]:
def num(col):
    try:
        return re.findall(patron_num, col)[0]
    except: 
        return np.nan

In [221]:
df.to_csv("attacks_pandas_8.csv")

In [222]:
df["age"] = df["age"].apply(num)

In [223]:
# Comprobamos los datos únicos:

df["age"].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '51',
       '61', '59', '42', '6', '27', '23', '29', '39', '24', '12', '26',
       '71', '43', '9', '44', '14', '62', '52', '38', '68', '16', '47',
       '63', '70', '41', '53', '7', '66', '45', '74', '64', '8', '56',
       '49', nan, '77', '84', '5', '67'], dtype=object)

In [224]:
len(df["age"].unique())

72

Hemos perdido más de veinte valores únicos, ¿tratamos de recuperarlos o podemos dar por satisfecha la limpieza?

In [225]:
df["age"] = pd.to_numeric(df["age"])

In [226]:
df.dtypes

case_number            object
year                    Int64
type                   object
country                object
area                   object
location               object
age                   float64
species_               object
date                   object
mes                    object
fatal_(y/n)_limpio     object
sex                    object
country_               object
dtype: object

3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [227]:
df.to_csv("datos/attacks_pandas_8.csv")